In [1]:
pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.1 MB/s eta 0:00:00


In [2]:
#All imports go into here ONLY.
from   sklearn.feature_extraction.text import CountVectorizer

from   sklearn.preprocessing   import StandardScaler
from   sklearn.pipeline        import Pipeline
from   sklearn.preprocessing   import OneHotEncoder
from   sklearn.preprocessing   import LabelEncoder
from   sklearn.utils           import sparsefuncs
from   sklearn.metrics         import f1_score
from   sklearn.decomposition   import PCA
from   sklearn.decomposition   import TruncatedSVD
from   sklearn.datasets        import load_iris
from   sklearn.linear_model    import Perceptron
from   sklearn.model_selection import train_test_split
from   sklearn.naive_bayes     import BernoulliNB
from   sklearn.naive_bayes     import MultinomialNB
from   sklearn.metrics         import accuracy_score, confusion_matrix
from   sklearn.model_selection import GridSearchCV
from   sklearn.pipeline        import make_pipeline
from   sklearn.metrics         import mean_squared_error
from   sklearn.metrics         import make_scorer

from   scipy                   import sparse
from   subprocess              import check_output

import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns
import pycountry         as pyctry

In [3]:
country_names = np.array( [ country.name for country in pyctry.countries ] )

raw_text_input_df = pd.read_excel(
                                    "/content/drive/MyDrive/TextFingerprinter/raw_text_input.xlsx",
                                    #true_values="Y",
                                    #false_values="N",
                                    #keep_default_na = False,
                                    #na_values = "nth",
                                    #index_col = "FirstName"
                    )

x_train, x_test, y_train, y_test = train_test_split(raw_text_input_df["RAW_TEXT"], raw_text_input_df["RESULT"], test_size=0.2, random_state=10)


country_vectorizer = CountVectorizer(

                                       #input      = x_train,
                                       lowercase  = False,
                                       vocabulary = country_names
)


pipeline_centered_SVD  = Pipeline( [

                                     ( "counter_vectorizer", CountVectorizer() ),
                                     (                "pca", PCA()             ),
                                     #(          "Bernoulli", BernoulliNB()     ),
                                 ]

                        )

pipeline_truncated_SVD  = Pipeline( [
                                      ( "counter_vectorizer", CountVectorizer() ),
                                      (      "truncated_SVD", TruncatedSVD()    ),

                                  ]

                         )

label_encoder          = LabelEncoder()

param_dict             =  {

                                 "counter_vectorizer__lowercase"  : False,
                                 "counter_vectorizer__vocabulary" : country_names,
                                "counter_vectorizer__ngram_range" : (1,4)

                        }

param_dict_trunc_SVD =  {

                                "counter_vectorizer__lowercase"  : False,
                                "counter_vectorizer__vocabulary" : country_names,
                               "counter_vectorizer__ngram_range" : (1,4),
                                   "truncated_SVD__n_components" : 3,

            }

result_array          = np.array( [ "Y" , "N" ] )

rmse = make_scorer(mean_squared_error, squared=False)

#print( y_train[0] )
#print( x_train[0] )

In [4]:
pipeline_centered_SVD.set_params(**param_dict)
countries_vectorized_pipeline = pipeline_centered_SVD.fit_transform( x_train )

print( countries_vectorized_pipeline[4] )

[ 1.23743687e+00  1.22714189e-33 -1.79448092e-33 -3.44812846e-49
  4.40870237e-50  1.91491614e-67  3.63478479e-67]


In [5]:
pipeline_truncated_SVD.set_params(**param_dict_trunc_SVD)
countries_vectorized_pipeline = pipeline_truncated_SVD.fit_transform( x_train )

print( countries_vectorized_pipeline[0] )
print( countries_vectorized_pipeline[4] )


[0. 0. 0.]
[1.41421356 0.         0.        ]


#Ensures the correctness of results_vectorized

In [31]:
for result in y_train:
  print( result )

label_encoder.fit( result_array )
results_vectorized      = label_encoder.transform( y_train )

print( result )

N
N
N
N
Y
N
N
N
N


In [13]:
print(len( countries_vectorized_pipeline ))
print(len( results_vectorized ))

8
8


In [42]:
model = BernoulliNB()
model.fit(
                      countries_vectorized_pipeline,
                      results_vectorized
)

x_test_transformed      = pipeline_truncated_SVD.transform( x_test )
y_test_transformed      = label_encoder.transform( y_test )

y_pred                  = model.predict( x_test_transformed )


print("-------F1 score------")
f1_score( y_test_transformed , y_pred, average="macro" )

0
0
True
-------F1 score------


1.0

In [65]:
#grid_search_score =

model = GridSearchCV(
                      estimator  = BernoulliNB(),
                      #param_grid = { 'alpha': [0.1, 0.5, 1.0, 2.0], 'binarize': [0.0, 0.5, 1.0]  },
                      param_grid = { },
                      n_jobs     = -1,
                      cv         = 5,
                      scoring      = "f1_macro"

        )

model.fit(
                      countries_vectorized_pipeline,
                      results_vectorized
)



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=BernoulliNB(), n_jobs=-1, param_grid={},
             scoring='f1_macro')

In [68]:
print("--------GridSearchCV_F1_Score-----------")
print(model.best_score_)

--------GridSearchCV_F1_Score-----------
0.8666666666666668


#PCA Analysis
When SVD == True